### Steganography on JPEG file

In [6]:
from PIL import Image
from bitarray import bitarray
import numpy as np

In [7]:
MESSAGE = "AATest"

def encode_message(msg):

    msg_bits = []

    msg_len = len(msg)
    msg_len_bytes = format(msg_len, '16b')

    msg_ascii = [ord(char) for char in msg]
    msg_bytes = [format(char, '08b') for char in msg_ascii]

    for bit in msg_len_bytes:
        if bit == ' ':
            bit = 0
            
        msg_bits.append(int(bit))

    for char in msg_bytes:
        for bit in char:
            msg_bits.append(int(bit))

    return msg_bits

encoded_message_bits = encode_message(MESSAGE)

In [53]:
IMG_DIR = "sample_image.jpg"

def is_even(val):
    return 1 if val % 2 == 0 else 0

def set_bit(value, bit):
    
    if is_even(value):
        if bit == 0:
            value += 1 #od razu zwracać
    else:
        if bit == 1:
            value += 1

    return value
    # return value

def encode_message_into_image(img, msg_bits):

    img = np.array(img)
    img_shape = img.shape
    img_flat = img.flatten()

    for i, bit in enumerate(msg_bits):
        
        val_index = i * 3
        value = img_flat[val_index]
        img_flat[val_index] = set_bit(value, bit)

    encoded_image = img_flat.reshape(img_shape)

    return Image.fromarray(encoded_image)
        


img = Image.open(IMG_DIR)

encoded_image = encode_message_into_image(img, encoded_message_bits)

In [64]:
def decode_message_from_image(img):
    msg = ""

    img = np.array(img)
    img_flat = img.flatten()

    len_bits = []

    for i in range(16):

        len_bits.append(is_even(img_flat[i * 3]))

    length = 0

    for idx, bit in enumerate(reversed(len_bits)):
        length |= bit << idx

    x = 0

    for char_index in range(length):

        char_val = 0


        for pixel_index in range(8):

            val_index = (16 * 3) + (x * 3)

            char_val |= is_even(img_flat[val_index]) << (7 - pixel_index)

            x += 1
        
        msg += chr(char_val)

    return msg

decode_message_from_image(encoded_image)

'AATest'